In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from typing_extensions import Literal, TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import Image, display
from agno.agent import Agent
from agno.tools.email import EmailTools
from agno.models.groq import Groq

# Load environment variables
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

# Initialize LLM
llm = ChatGroq(model="qwen-2.5-32b")

# Define routing structure
class Route(BaseModel):
    step: Literal["support team", "technical team", "marketing team"] = Field(
        None, description="The next step of the routing process"
    )

router = llm.with_structured_output(Route)

class State(TypedDict):
    input: str
    decision: str
    output: str

# Define team functions
def support(state: State):
    """Handles routing to the Support Team."""
    return {"output": "It's sent to Support Team"}

def tech(state: State):
    """Handles routing to the Technical Team."""
    return {"output": "It's sent to Technical Team"}

def marketing(state: State):
    """Handles routing to the Marketing Team."""
    return {"output": "It's sent to Marketing Team"}

def llm_call_router(state: State):
    """Determines the routing decision using the LLM."""
    decision = router.invoke([
        SystemMessage(content=(
            "You are an AI assistant responsible for routing emails to the correct team. Analyze the email content and determine "
            "whether it should be sent to the Support Team, Technical Team, or Marketing Team. Use the following criteria:\n\n"
            "- **Support Team**: Handles customer service inquiries, order issues, and general support requests.\n"
            "- **Technical Team**: Resolves technical problems such as system errors, login issues, and software bugs.\n"
            "- **Marketing Team**: Manages promotional campaigns, collaborations, and brand partnerships."
        )),
        HumanMessage(content=state['input'])
    ])
    return {'decision': decision.step}

def router_decision(state: State):
    """Determines the next step based on the LLM decision."""
    if state['decision'] == "support team":
        return "support"
    elif state['decision'] == "technical team":
        return "tech"
    elif state['decision'] == "marketing team":
        return "marketing"

# Build the workflow graph
builder = StateGraph(State)

builder.add_node("Support_Team", support)
builder.add_node("Tech_Team", tech)
builder.add_node("Marketing_Team", marketing)
builder.add_node("llm_call_router", llm_call_router)

builder.add_edge(START, "llm_call_router")
builder.add_conditional_edges(
    "llm_call_router",
    router_decision,
    {
        "tech": "Tech_Team",
        "support": "Support_Team",
        "marketing": "Marketing_Team"
    }
)
builder.add_edge("Support_Team", END)
builder.add_edge("Tech_Team", END)
builder.add_edge("Marketing_Team", END)

router_workflow = builder.compile()
# display(Image(router_workflow.get_graph().draw_mermaid_png()))

# Example email input
response = router_workflow.invoke({
    "input": """I recently placed an order (#12345) on your website, but I haven't received any updates regarding the shipping status. The estimated delivery date has passed, and the tracking link isn't showing any details.

Could you please check the status and provide an update?"""
})

# Define Agno Agent for sending emails
receiver_email = os.getenv('receiver_email')
sender_email = os.getenv('sender_email')
sender_name = "Krish"
sender_passkey = os.getenv('sender_passkey')

agent = Agent(
    model=Groq(id="qwen-2.5-32b"),
    instructions=[
        "You are tasked with crafting and sending an email response based on the following:",
        f"1. The email content received from the sender: {response['input']}",
        f"2. The team responsible for responding: {response['output']}",
        "Compose a professional email addressing the sender's concerns and include the team responsible for the response in the signature.",
    ],
    tools=[
        EmailTools(
            receiver_email=receiver_email,
            sender_email=sender_email,
            sender_name=sender_name,
            sender_passkey=sender_passkey,
        )
    ],
)

# Send email
agent.print_response(f"send an email to {receiver_email}")


Output()

INFO Sending Email to sigmamale2404@gmail.com

![alt text](../images/email.png)